In [41]:
# python degrtytaee/generate_data_degree_e2e.py -c config/config_degree_e2e_ace05e.json
# python degree/train_degree_e2e.py -c config/config_degree_e2e_ace05e.json

In [1]:
import copy
import re
import json
import os
import random
from tqdm import tqdm, trange

import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import nltk
import spacy
# nltk.download('wordnet')
# nltk.download('punkt_tab')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize 
from transformers import BertTokenizer
import multiprocessing

nlp = spacy.load("en_core_web_sm")
bert_tokenizer = BertTokenizer.from_pretrained('bert-large-cased', do_lower_case=False)

D:\Function\Anoconda3\envs\data\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
from utils.util import *

In [5]:
def cut_mention(word):
    # black_list = ['from', 'an', 'a', 'the', 'to', 'during', 'on', 'after', 'as', 'near', 'before']
    doc = nlp(word)
    word_prop = [i.pos_ for i in doc]

    if len(word_prop) == 1:
        return word
    if len(word_prop) == 2:
        if word_prop[0] in ['DET', 'ADJ', 'NUM']:
            word = ' '.join(word.split(' ')[1:])
    elif len(word_prop) >= 2:
        if word_prop[0] in ['DET', 'ADJ', 'NUM'] and word_prop[1] in ['DET', 'ADJ', 'NUM']:
            word = ' '.join(word.split(' ')[2:])
    # word_repl = ''
    # for i in word.split():
    #     if i not in black_list:
    #         word_repl += i
    return word



In [7]:
r_e = {}
for fn in os.listdir('./meta_data/arg_roles/'):
    if not fn.endswith('.txt'):
        continue
    with open('./meta_data/arg_roles/' + fn, 'r', encoding='utf-8') as f:
        r_data = json.load(f)
    for k in r_data:
        e_t = (fn.split('.txt')[0] + ":" + k).upper()
        r_e[e_t] = {}
        for role in r_data[k]:
            r_e[e_t][role.replace('-Arg', '')] = r_data[k][role][0].split('/')

In [9]:
# data = {}
# for fn in os.listdir('./'):
#     if fn.endswith('.json') and re.search('^data_\d*?_', fn):
#         with open('./' + fn, 'r', encoding='utf-8') as f:
#             r_data = json.load(f)
#         # print(fn)
#         for k, v in r_data.items():
#             data[k] = v
#         #     print(k, len(v))
#         # print()
# with open('./checkpoint.json', 'w', encoding='utf-8') as f:
#     json.dump(data, f)

In [145]:
event_dict = load_json('./meta_data/event_dict_full.json')
# practical_roles = load_pickle('./exp_data/practical_roles.pkl')

In [147]:
with open('./data_sen_7_29_7_37.json', 'r', encoding='utf-8') as f:
    all_data = json.load(f)

In [151]:
with open('./exp_data/trigger_sout_top10.json', 'r', encoding='utf-8') as f:
    trigger_pool = json.load(f)

In [153]:
def unify_arg_record(sen, reco):
    for e_id in reco:
        for role in reco[e_id]['argument']:
            for i, arg in enumerate(reco[e_id]['argument'][role]):
                if re.search(arg.lower(), sen.lower(), re.S) and not re.search(arg, sen, re.S):
                    indices = [[match.start(), match.end()] for match in re.finditer(arg.lower(), sen.lower())]
                    start, end = indices[0]
                    arg_repl = sen[start:end]
                    reco[e_id]['argument'][role][i] = arg_repl
    return reco

In [155]:
def search_upper(sen, w, event):
    w_lower = w.lower()
    sen_lower = sen.lower()
    if re.search(w, sen, re.S):
        return sen
    elif re.search(w_lower, sen_lower, re.S):
        indices = [[match.start(), match.end()] for match in re.finditer(w.lower(), sen.lower())]
        for start, end in indices:
            sen = sen[:start] + w + sen[end:]
        return sen
    else:
        return sen

def parse_capital(sen, rec):
    try:
        event = rec['event']
        tri = rec['trigger']
        args = rec['argument']
        if not re.search(tri, sen):
            sen = search_upper(sen, tri, event)
        for k, v in args.items():
            for a in v:
                sen = search_upper(sen, a, event)
        
        return sen
    except:
        # print('error:\tindex.{}\t{}'.format(sen, rec))
        return ''

In [157]:
t = 0
for k, v in all_data.items():
    for i, r in enumerate(all_data[k]):
        sen = r['sentence'] 
        all_data[k][i]['sentence'] = {}
        all_data[k][i]['sentence']['sentences'] = [sen]
        pop_list = []
        for event_id in all_data[k][i]['record']:
            event = all_data[k][i]['record'][event_id]['event']
            all_data[k][i]['record'][event_id]['argument'] = {}
            try:
                real_role = [role for role in event_dict[event][1]]
            except:
                pop_list.append(event_id)
        for pop_event in pop_list:
            _ = all_data[k][i]['record'].pop(pop_event)


In [161]:
def parse_arglist(args):
    new_arg = []
    for arg in args:
        if isinstance(arg, list):
            new_arg.extend(arg)
        else:
            new_arg.append(arg)
    return Series(new_arg).drop_duplicates().tolist()
    
for e in all_data:
    for i in range(len(all_data[e])):
        sen = all_data[e][i]['sentence']
        record = all_data[e][i]['record']
        for e_id in record:
            for role in record[e_id]['argument']:
                args = parse_arglist(record[e_id]['argument'][role])
                record[e_id]['argument'][role] = args
        all_data[e][i]['record'] = record


In [163]:
sen_event = {}
for e in all_data:
    sen_event[e] = []
    for i in range(len(all_data[e])):
        sen = all_data[e][i]['sentence']
        record = all_data[e][i]['record']
        for e_id in record:
            for role in record[e_id]['argument']:
                args = [cut_mention(arg) for arg in record[e_id]['argument'][role]]
                # args = [arg for arg in record[e_id]['argument'][role]]
                
                record[e_id]['argument'][role] = args
        sen_event[e].append([' '.join(sen['sentences']), record])


In [165]:
event_sen_pair = []
for e in sen_event:
#     [sen, rec]
    for i in range(len(sen_event[e])):
        sentence = sen_event[e][i][0]
        for record in sen_event[e][i][1]:
            r = sen_event[e][i][1][record]
            sentence = parse_capital(sentence, r)
        rec = unify_arg_record(sentence, sen_event[e][i][1])
        if sentence:
            event_sen_pair.append({'sentence':sentence, 'record': rec})
        else:
            continue

In [167]:
from pandas import Series

In [169]:
s = 0
for ep in event_sen_pair:
    sen = ep['sentence']
    if len(re.findall('trigger', sen)) != len(ep['record']):
        s += 1
print(0)

0


In [171]:
event_sen_pair_refined = []
for s_r in event_sen_pair:
    sentence = s_r['sentence']
    pop_list = []
    for event_id in s_r['record']:
        e = s_r['record'][event_id]['event']
        trigger = s_r['record'][event_id]['trigger']
        if not re.findall(trigger, sentence, flags=re.I):
            pop_list.append(event_id)
    for pop_event in pop_list:
        _ = s_r['record'].pop(pop_event)

    event_sen_pair_refined.append({'sentence':sentence, 'record': s_r['record']})

In [173]:
tri_mis = {}
arg_mis = {}
for i, d in enumerate(event_sen_pair_refined):
    try:
        sen = d['sentence']
        rec = d['record']
        if not i in tri_mis.keys():
            tri_mis[i] = []
        if not i in arg_mis.keys():
            arg_mis[i] = []
        for _, e in rec.items():
            tri = e['trigger']
            args = e['argument']
            event = e['event']
            if not re.search(tri, sen):
                tri_mis[i].append([tri, sen])
            for k, v in args.items():
                for a in v:
                    if not re.search(a, sen):
                        arg_mis[i].append({k:[v, sen]})
                    if len(a.split()) > 4:
                        # print({k:[v, sen]})
                        arg_mis[i].append({k:[v, sen]})
    except:
        print('error:\tindex.{}\t{}'.format(i, d))

In [175]:
s = 0
for k, v in tri_mis.items():
    if v:
        # print(v)
        # print(event_sen_pair_refined[k])
    
        s+=1
print(s)

112


In [177]:
temp_list = []
sum_ = 0
for k, v in arg_mis.items():
    if v:
        # print(k, v)
        # break
        sum_ += 1
    else:
        temp_list.append(event_sen_pair_refined[k])
event_sen_pair_refined = temp_list

In [179]:
print(sum_)

0


In [181]:
len(event_sen_pair_refined)

1650

In [183]:
for i in range(len(event_sen_pair_refined)):
    es_p = event_sen_pair_refined[i]
    for event_id in es_p['record']:
        for e in r_e.keys(): 
            if e.endswith(es_p['record'][event_id]['event'].upper()):
                event_sen_pair_refined[i]['record'][event_id]['event'] = e
                break
                
eve_se_p_f = []
for es_p in event_sen_pair_refined:
    if len(es_p['record']) < 1 or len(es_p['record']) > 5:
        continue
    for event_id in es_p['record']:
        if len(es_p['record'][event_id]['trigger'].split(' ')) > 1:
            continue
    eve_se_p_f.append(es_p)

In [185]:
# for i in range(len(event_sen_pair_refined)):
#     sen = event_sen_pair_refined[i]['sentence']
#     r = event_sen_pair_refined[i]['record']['event_1']
#     e = r['event']
#     trigger = r['trigger']
#     if len(trigger.split(' ')) < 2:
#         continue
#     p = 0
#     for sub_t in trigger.split(' '):
#         for t in trigger_pool[e]:
#             if len(t.split(' ')) > 1:
#                 continue
#             if sub_t.upper() == t.upper():
# #                 if len(t) < 3:
# #                     print(s_data[e][i]['record'][r]['trigger'], '\t', sub_t, '\t', t)
#                 event_sen_pair_refined[i]['record']['event_1']['trigger'] = sub_t
#                 p = 1
#                 break
#         if p == 1:
#             break

In [187]:
# wnl = WordNetLemmatizer()
# for i in range(len(eve_se_p_f)):
#     es_p = eve_se_p_f[i]
#     sen_l = es_p['sentence'].split(' ')
#     trigger = es_p['record']['event_1']['trigger']
#     for w in sen_l:
#         if wnl.lemmatize(trigger.strip(), 'v') == wnl.lemmatize(w.strip('[]().,'), 'v'):
#             eve_se_p_f[i]['record']['event_1']['trigger'] = w.strip('[]().,')
#             break


In [189]:
# cnt = 0
# for i in eve_se_p_f:
#     sen = i['sentence']
#     t = i['record']['event_1']['trigger']
    
#     if not len(re.findall(t, sen)):
#         cnt += 1

# cnt

In [191]:
def capitalize_event(s):
    s_l = s.strip().split(':')
    f_e = s_l[0].capitalize()
    s_ee = ''
    for s_e in s_l[1].split('-'):
        s_ee += s_e.capitalize() + '-'
    return f_e + ':' + s_ee[:-1]

def convert_format(input_data, bert_tokenizer, role_to_entity_types):
    # 初始化BERT tokenizer
    timestamp = f"{random.randint(2000, 2022)}{random.randint(10,12):02}{random.randint(10,31):02}"
    doc_id = f"CNN_ENG_{timestamp}_{random.randint(100000, 999999)}.1"
    wnd_id = f"{doc_id}-4"

    # 处理原始tokens和子词分词
    sentence = input_data["sentence"]
    raw_tokens = word_tokenize(sentence)
    
    # 生成pieces和token_lens
    pieces, token_lens = [], []
    for token in raw_tokens:
        subwords = bert_tokenizer.tokenize(token)
        pieces.extend(subwords)
        token_lens.append(len(subwords))

    # 构建entity_mentions
    entity_mentions = []
    used_texts = set()

    # 遍历所有事件参数
    for event in input_data["record"].values():
        for role, texts in event["argument"].items():
            for text in texts:
                if text in used_texts:
                    continue  # 避免重复创建相同text的实体
                used_texts.add(text)

                # 查找文本在tokens中的位置
                text_tokens = text.split()
                start = -1
                for i in range(len(raw_tokens) - len(text_tokens) + 1):
                    if raw_tokens[i:i+len(text_tokens)] == text_tokens:
                        start = i
                        end = i + len(text_tokens)
                        break

                if start == -1:
                    continue  # 未找到对应位置

                # 获取实体类型（取第一个匹配类型）
                entity_type = role_to_entity_types.get(event['event'].upper()).get(role, ["UNK"])[0]

                entity_mentions.append({
                    "id": f"{wnd_id}-E{len(entity_mentions)}",
                    "start": start,
                    "end": end,
                    "entity_type": entity_type,
                    "mention_type": "UNK",
                    "text": text
                })

    # 构建event_mentions
    event_mentions = []
    for ev_idx, event in enumerate(input_data["record"].values()):
        arguments = []
        for role, texts in event["argument"].items():
            for text in texts:
                # 查找对应的实体
                entity = next(
                    (e for e in entity_mentions if e["text"] == text),
                    None
                )
                if entity:
                    arguments.append({
                        "entity_id": entity["id"],
                        "text": text,
                        "role": role
                    })

        # 查找trigger位置
        trigger_tokens = event["trigger"].split()
        trigger_start = -1
        for i in range(len(raw_tokens) - len(trigger_tokens) + 1):
            if raw_tokens[i:i+len(trigger_tokens)] == trigger_tokens:
                trigger_start = i
                trigger_end = i + len(trigger_tokens)
                break

        event_mentions.append({
            "event_type": capitalize_event(event["event"]),
            "id": f"{wnd_id}-EV{ev_idx}",
            "trigger": {
                "start": trigger_start,
                "end": trigger_end,
                "text": event["trigger"]
            },
            "arguments": arguments
        })

    return {
        "doc_id": doc_id,
        "wnd_id": wnd_id,
        "entity_mentions": entity_mentions,
        "relation_mentions": [],
        "event_mentions": event_mentions,
        "entity_coreference": [],
        "event_coreference": [],
        "tokens": raw_tokens,
        "pieces": pieces,
        "token_lens": token_lens,
        "sentence": sentence,
        "sentence_starts": [0]
    }



In [193]:
#同位语修饰词，定语共指 -》 删除complexity control， 增加一事件比重
#不合理的搭配 in iraq, Anwar submitted his resignation from United States
# chain -> 提高arg f1, 降低trigger f1
# constrain -> 有效
# faiss sen * 10 + chain -> 略微降低arg f1, 略微提高trigger f1(0.5)
# faiss sens[:4] + chain -> trigger f1 + 5, arg f1 - 1，无效
# 核心 -》 有效记录 -》 模板生成 + faiss ? 约束解码训练？
# 'Org': ['Stop n Shop'] -> 不合理arg

# to do -> 1.删除complexity control，chain，添加faiss sen * 10 2.增加一事件比重

# 提升：1.抽取
#      2.constrain
#      3.模板生成
eve_se_p_f[:10]

[{'sentence': 'After the elections concluded, the newly elected leaders attended a peace summit to address the ongoing fighting in the region, which had resulted in numerous injuries among civilians.',
  'record': {'event_1': {'event': 'PERSONNEL:ELECT',
    'trigger': 'elections',
    'argument': {}},
   'event_2': {'event': 'CONTACT:MEET', 'trigger': 'summit', 'argument': {}},
   'event_3': {'event': 'CONFLICT:ATTACK',
    'trigger': 'fighting',
    'argument': {}},
   'event_4': {'event': 'LIFE:INJURE',
    'trigger': 'injuries',
    'argument': {}}}},
 {'sentence': 'After winning the election, the newly appointed mayor was shot during a violent protest, just hours before he was scheduled to meet with city officials; meanwhile, the governor was calling other leaders to coordinate a response.',
  'record': {'event_1': {'event': 'PERSONNEL:ELECT',
    'trigger': 'Election',
    'argument': {}},
   'event_2': {'event': 'CONTACT:MEET', 'trigger': 'meet', 'argument': {}},
   'event_3': {

In [195]:
# 测试运行
# 示例映射字典


ROLE_ENTITY_MAPPING = r_e
train_data = eve_se_p_f

data = []
cnt = 0

for input_data in tqdm(train_data, desc='Processing'):
    try:
        output = convert_format(input_data, bert_tokenizer, ROLE_ENTITY_MAPPING)
        data.append(output)
    except:
        # print(input_data)
        # break
        cnt += 1

print(cnt)

Processing: 100%|█████████████████████████████████████████████████████████████████| 1598/1598 [00:07<00:00, 220.36it/s]

119


In [199]:
d = {i:0 for i in range(10)}
for r in eve_se_p_f:
    if len(r['record']) < 1:
        print(r)
    d[len(r['record'])] += 1 / len(eve_se_p_f)
d

{0: 0,
 1: 0.10262828535669574,
 2: 0.08948685857321648,
 3: 0.30788485607008875,
 4: 0.30100125156445645,
 5: 0.1989987484355437,
 6: 0,
 7: 0,
 8: 0,
 9: 0}

In [201]:
print(cnt)

119


In [203]:
len(data)

1479

In [205]:
with open('./exp_data/train.w1.oneie.json', 'w', encoding='utf-8') as f1:
    for l in data:
        d = json.dumps(l)
        f1.write(d + '\n')

In [207]:
data[0]

{'doc_id': 'CNN_ENG_20101123_281932.1',
 'wnd_id': 'CNN_ENG_20101123_281932.1-4',
 'entity_mentions': [],
 'relation_mentions': [],
 'event_mentions': [{'event_type': 'Personnel:Elect',
   'id': 'CNN_ENG_20101123_281932.1-4-EV0',
   'trigger': {'start': 2, 'end': 3, 'text': 'elections'},
   'arguments': []},
  {'event_type': 'Contact:Meet',
   'id': 'CNN_ENG_20101123_281932.1-4-EV1',
   'trigger': {'start': 12, 'end': 13, 'text': 'summit'},
   'arguments': []},
  {'event_type': 'Conflict:Attack',
   'id': 'CNN_ENG_20101123_281932.1-4-EV2',
   'trigger': {'start': 17, 'end': 18, 'text': 'fighting'},
   'arguments': []},
  {'event_type': 'Life:Injure',
   'id': 'CNN_ENG_20101123_281932.1-4-EV3',
   'trigger': {'start': 27, 'end': 28, 'text': 'injuries'},
   'arguments': []}],
 'entity_coreference': [],
 'event_coreference': [],
 'tokens': ['After',
  'the',
  'elections',
  'concluded',
  ',',
  'the',
  'newly',
  'elected',
  'leaders',
  'attended',
  'a',
  'peace',
  'summit',
  'to

In [79]:
# train = data
# sen_cnt = 0
# event_data = {}
# c_m = 0
# for data_ in train:
#     # if len(data['event_mentions']) > 1:
#     #     c_m += 1
#     #     e_type = e['event_type']

#     for e in data_['event_mentions']:
#         e_type = e['event_type']
#         sen_cnt += 1
#         if not event_data.get(e_type):
#             event_data[e_type] = []
#         for info in e['arguments']:
#             if info['role'] not in event_data[e_type]:
#                 event_data[e_type].append(info['role'])